# 05: PCA Visualization and Brief

## Objectives

- PCA to 2D for visualization only; do not use it to choose k.
- Create a clean scatter with clusters colored; add convex hulls or centroids.
- Write a one-page brief summarizing:
  - k choice and metric values
  - names + sizes of segments
  - 2–3 actionable ideas per segment


In [ ]:
# === TODO ===
# PCA to 2D on X_ready; scatter colored by cluster; centroids marked.
# Acceptance: PNG saved to artifacts/reports/pca_clusters.png


In [ ]:
# === TODO ===
# Write brief to artifacts/reports/brief.md with key numbers and segment names.
# Hints: use Python to open(...) and write Markdown
# Acceptance: brief.md exists with > 12 lines summarizing findings
